# Topic :  Open API and AI Agents Code-Along

Adopted from [Colab Notebook](https://colab.research.google.com/drive/1xYq4wr4dkmvOuq0Ljwt1W9fIxfR50ekq)

This notebook has 4 main sections:
* Using AgentWorkflow
* Building a multi-agent system with AgentWorkflow
* Building agentic Workflows
* Building a multi-agent system with Workflows

In LlamaIndex, `Workflow`s are the building blocks of putting together an agent or a multi-agent system. To get you up and running quickly, we have a pre-built Workflow called the `AgentWorkflow` that handles a lot of common agent use cases and allows you to focus on the specifics of your implementation. Later, we drop down one level to Workflows themselves and show how to build one from scratch.

To start off, we'll create an AgentWorkflow with a single agent, and use that to cover the basic functionality of the `AgentWorkflow` class.

In [1]:
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print("OPENAI_API_KEY:", OPENAI_API_KEY[:10])
print("TAVILY_API_KEY:", TAVILY_API_KEY[:10])

OPENAI_API_KEY: sk-proj-Cq
TAVILY_API_KEY: tvly-dev-x


With our API key in hand, instantiating an LLM is one line...

In [2]:
from llama_index.llms.openai import OpenAI
llm2 = OpenAI(model="o4-mini-2025-04-16", api_key=OPENAI_API_KEY, max_tokens=100000)
llm = OpenAI(model="gpt-4.1-mini-2025-04-14", api_key=OPENAI_API_KEY)
response = llm.complete("What did I ask you before?")
print(response)

I don’t have access to past interactions in this conversation. How can I assist you today?


## Building a tool

A core feature of agents is that they can use tools to find out more about the state of the world, or take action in response to instructions, without you needing to be explicit about how and when they do that.

In this example, we'll give our agent the ability to search the web for information by creating a tool that does that. We'll use a service called Tavily, which is specifically designed to provide this kind of tool to agents. You can get a free API key from [Tavily](https://tavily.com/).

In [ ]:
from tavily import AsyncTavilyClient

# note the type annotations for the incoming query and the return string
# Performs a basic web search using Tavily and returns the result as a string.
async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=TAVILY_API_KEY)
    return str(await client.search(query))

# Performs an advanced web search with more depth and results, returns the result as a string.
async def advanced_search_web(query: str) -> dict:
    """Useful for performing in-depth web searches web to answer questions."""
    client = AsyncTavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    return str(
        await client.search(
            query=query,
            search_depth="advanced",
            max_results=20,
            include_answer="advanced",
        )
    )

# Crawls a website starting from the base_url, following links up to a certain depth and breadth.
async def crawl_web(base_url: str) -> str:
    """Useful for using the looking up URL for citations"""
    client = AsyncTavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    results = await client.crawl(
        url=base_url,
        max_depth=3,
        limit=20,
        max_breadth=20,
        extract_depth="advanced",
        select_domains=[""],
        allow_external=True,
    )
    return results

# Extracts text content from a given URL using Tavily's extraction capabilities.
async def extract_text_from_url(url: str) -> dict:
    """Extract text from a URL."""
    client = AsyncTavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    response = await client.extract(urls=[url], extract_depth="advanced")
    return response

In [ ]:
await advanced_search_web("Types of coffee beans")

In [3]:
from semantic_search import search_local_corpus

## Instantiating an AgentWorkflow

With the tool and and LLM defined, we can create an `AgentWorkflow` that uses the tool. AgentWorkflow has a special helper method for creating a single agent from a set of tools, so we'll use that.

We give it a system prompt that defines what the agent does. It's a good idea to tell the agent what kinds of things its tools will allow it to do.

In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    [search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that answers questions. If you don't know the answer, you can search the web for information. Format your response in markdown, prefer tables, and include citations for any information you find online.",
)

In [5]:
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    [search_local_corpus],
    llm=llm,
    system_prompt="You are a helpful assistant that answers questions from the locally stored oncology handbook. If you don't know the answer, tell it clearly. Format your response in markdown, prefer tables, and include citations for any information you find online.",
)

## Running the Agent

Now that our agent is created, we can run it! An AgentWorkflow expects to start with a question or prompt in the `user_msg`, which it passes to the agent.

In [6]:
from IPython.display import display, Markdown
response = await workflow.run(user_msg="What are the relevant genes for colorectal cancer?")
display(Markdown(str(response)))

The relevant genes commonly associated with colorectal cancer include:

| Gene       | Role/Significance                                                                                  |
|------------|--------------------------------------------------------------------------------------------------|
| APC        | Tumor suppressor gene mutated in about 70-80% of colorectal cancers; involved in Wnt signaling pathway regulation. |
| KRAS       | Oncogene mutated in colorectal cancer; mutations lead to uncontrolled cell growth.                |
| TP53       | Tumor suppressor gene frequently mutated in colorectal cancer; involved in apoptosis and DNA repair. |
| Mismatch Repair Genes (MLH1, MSH2, MSH6, PMS2) | Mutations cause microsatellite instability and are linked to hereditary non-polyposis colorectal cancer (Lynch syndrome). |
| BRAF       | Oncogene; mutations can be involved in colorectal cancer progression.                             |
| PTEN       | Tumor suppressor gene; loss leads to activation of growth factor signaling pathways.              |

Additional notes:
- APC mutations are often the initiating event in colorectal tumorigenesis.
- KRAS mutations occur in a significant subset and predict poor prognosis and resistance to certain therapies (e.g., anti-EGFR therapy).
- Microsatellite instability due to mismatch repair gene defects is a hallmark of a subset of colorectal cancers and is associated with better prognosis but lack of benefit from fluorouracil adjuvant therapy.
- TP53 mutations contribute to tumor progression.
- BRAF mutations are also considered in molecular profiling.
- Genetic predisposition syndromes such as Familial Adenomatous Polyposis (FAP) and Lynch syndrome involve mutations in APC and mismatch repair genes respectively.

References:
- The information is based on the oncology handbook chapters on colorectal cancer genetics and molecular biology.
- [Adam et al., The Oncosurgery Approach to Managing Liver Metastases from Colorectal Cancer, Oncologist 2013]
- [General oncology genetics chapters on tumor suppressor genes and oncogenes]

If you need detailed information on any specific gene or its clinical implications, please let me know!

## Maintaining State

By default, the `AgentWorkflow` is stateless between runs. This means that the agent will not have any memory of previous runs.

To maintain state, we need to keep track of the previous state. In LlamaIndex, Workflows have a `Context` that can be used to maintain state within and between runs. Since the `AgentWorkflow` is just a pre-built `Workflow`, we can also use it now.

To maintain state between runs, we'll create a new `Context` called `ctx`. We pass in our `workflow` to properly configure this Context object for the workflow that will use it.

With our configured Context, we can pass it to our first run.

In [ ]:
from llama_index.core.workflow import Context

# configure a context to work with our workflow
ctx = Context(workflow)

response = await workflow.run(
    user_msg="My name is Rangan, nice to meet you!", ctx=ctx # give the configured context to the workflow
)
print(str(response))

Now we can pass the same context to a second run, and it will remember what happened before:

In [ ]:
# run the workflow again with the same context
response = await workflow.run(user_msg="What is my name?", ctx=ctx)
print(str(response))

You can save and load the agent's context to maintain memory over the longer term. By serializing the `Context` object to disk and restoring it later, the agent can remember previous interactions even after the notebook or process is restarted. This enables persistent memory across sessions.

In [ ]:
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer
import json

ctx_dict = ctx.to_dict(serializer=JsonSerializer())
with open("ctx_dict.json", "w") as f:
    json.dump(ctx_dict, f)


In [ ]:
with open("ctx_dict.json", "r") as f:
    ctx_dict = json.load(f)
restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

In [ ]:
response3 = await workflow.run(user_msg="What's my name?", ctx=restored_ctx)
print(str(response3))

## Accessing Context from within tools

Tools can also be defined that have access to the workflow context. This means you can set and retrieve variables from the context and use them in the tool or between tools.

`AgentWorkflow` uses a context variable called `state` that gets passed to every agent. You can rely on information in `state` being available without explicitly having to pass it in.

**Note:** To access the `Context`, the Context parameter should be the first parameter of the tool.

In [ ]:
from llama_index.core.workflow import Context

# we've set the first parameter to be of type Context
async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.get("state")
    state["name"] = name
    await ctx.set("state", state)
    return f"Name set to {name}"


stateful_workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

stateful_workflow_context = Context(stateful_workflow)

# this workflow will save data to the `state` variable in the context
response = await stateful_workflow.run(user_msg="My name is Rangan", ctx=stateful_workflow_context)
print(str(response))

# we can retrieve the value of `state` from the context directly
state = await stateful_workflow_context.get("state")
print("Name as stored in state: ",state["name"])

## Streaming output and events

In addition to returning a single string when the workflow completes, the `AgentWorkflow` also supports streaming. Since the `AgentWorkflow` is a `Workflow`, it can be streamed like any other `Workflow`. This works by using the handler that is returned from the workflow. The stream returns a variety of event types as the workflow executes, and you can select which ones to handle.

* If you want to stream the LLM output, you can use the `AgentStream` events, which contain a `delta` of the new output each time
* `AgentInput` events will tell you which agent is running (our current workflow just has one agent)
* `AgentOutput` events will tell you what the agents returned, including which tools they called
* `ToolCall` and `ToolCallResults` will track tools as they are called and their outputs

In this example we're only handling the `AgentStream` events, but you can see in the commented-out code how we'd handle the other kinds. You can tell our handling is working because the output will appear in chunks as you run the cell, rather than appearing all at once.

In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = workflow.run(user_msg="What is the weather in Saskatoon? Give detailed results in the form of a table")

async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)
        # print(event.response)  # the current full response
        # print(event.raw)  # the raw llm api response
        # print(event.current_agent_name)  # the current agent name
    # elif isinstance(event, AgentInput):
    #    print(event.input)  # the current input messages
    #    print(event.current_agent_name)  # the current agent name
    # elif isinstance(event, AgentOutput):
    #    print(event.response)  # the current full response
    #    print(event.tool_calls)  # the selected tool calls, if any
    #    print(event.raw)  # the raw llm api response
    # elif isinstance(event, ToolCallResult):
    #    print(event.tool_name)  # the tool name
    #    print(event.tool_kwargs)  # the tool kwargs
    #    print(event.tool_output)  # the tool output
    # elif isinstance(event, ToolCall):
    #     print(event.tool_name)  # the tool name
    #     print(event.tool_kwargs)  # the tool kwargs

In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = workflow.run(user_msg="What is the weather in Saskatoon?")

async for event in handler.stream_events():
    if isinstance(event, AgentInput):
       print("Agent input: ", event.input)  # the current input messages
       print("Agent name:", event.current_agent_name)  # the current agent name
    elif isinstance(event, AgentOutput):
       print("Agent output: ", event.response)  # the current full response
       print("Tool calls made: ", event.tool_calls)  # the selected tool calls, if any
       print("Raw LLM response: ", event.raw)  # the raw llm api response
    elif isinstance(event, ToolCallResult):
       print("Tool called: ", event.tool_name)  # the tool name
       print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
       print("Tool output: ", event.tool_output)  # the tool output

# Multi-agent System with AgentWorkflow

Our system will have three agents:

1. A `ResearchAgent` that will search the web for information on the given topic.
2. A `WriteAgent` that will write the report using the information found by the `ResearchAgent`.
3. A `ReviewAgent` that will review the report and provide feedback.

We will use the `AgentWorkflow` class to create a multi-agent system that will execute these agents in order.

There are a lot of ways we could go about building a system to perform this task. In this example, we will use a few tools to help with the research and writing processes.

1. A `web_search` tool to search the web for information on the given topic.
2. A `record_notes` tool which will save research found on the web to the state so that the other tools can use it.
3. A `write_report` tool to write the report using the information found by the `ResearchAgent`
4. A `review_report` tool to review the report and provide feedback.

Utilizing the `Context` class, we can pass state between agents, and each agent will have access to the current state of the system.

In [ ]:
async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=TAVILY_API_KEY)
    return str(await client.search(query))

async def record_notes(ctx: Context, notes: str, notes_title: str) -> str:
    """Useful for recording notes on a given topic."""
    current_state = await ctx.get("state")
    if "research_notes" not in current_state:
        current_state["research_notes"] = {}
    current_state["research_notes"][notes_title] = notes
    await ctx.set("state", current_state)
    return "Notes recorded."

async def write_report(ctx: Context, report_content: str) -> str:
    """Useful for writing a report on a given topic."""
    current_state = await ctx.get("state")
    current_state["report_content"] = report_content
    await ctx.set("state", current_state)
    return "Report written."

async def review_report(ctx: Context, review: str) -> str:
    """Useful for reviewing a report and providing feedback."""
    current_state = await ctx.get("state")
    current_state["review"] = review
    await ctx.set("state", current_state)
    return "Report reviewed."

With our tools defined, we can now create our agents.

If the LLM you are using supports tool calling (most big models do), you can use the `FunctionAgent` class, which is a little more efficient. Otherwise, you can use the `ReActAgent` class.

Here, the name and description of each agent is used so that the system knows what each agent is responsible for and when to hand off control to the next agent.

Like our single-agent system did before, each agent takes a `system_prompt` that tells it what it should do, and suggests how to work with the other agents.

You can also optionally help your multi-agent system constrain itself by listing which other agents an agent can talk to using `can_handoff_to` (otherwise it will just try to figure this out on its own).

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Useful for searching the web for information on a given topic and recording notes on the topic.",
    system_prompt=(
        "You are the ResearchAgent that can search the web for information on a given topic and record notes on the topic. "
        "Once notes are recorded and you are satisfied, you should hand off control to the WriteAgent to write a report on the topic."
    ),
    llm=llm,
    tools=[advanced_search_web, record_notes],
    can_handoff_to=["WriteAgent"],
)

write_agent = FunctionAgent(
    name="WriteAgent",
    description="Useful for writing a report on a given topic.",
    system_prompt=(
        "You are the WriteAgent that can write a report on a given topic. "
        "Your report should be in a markdown format. The content should be grounded in the research notes. Give citations for any information you find online. Provide URLs."
        "Once the report is written, you should get feedback at least once from the ReviewAgent."
    ),
    llm=llm,
    tools=[write_report],
    can_handoff_to=["ReviewAgent", "ResearchAgent"],
)

review_agent = FunctionAgent(
    name="ReviewAgent",
    description="Useful for reviewing a report and providing feedback.",
    system_prompt=(
        "You are the ReviewAgent that can review a report and provide feedback. "
        "Your feedback should either approve the current report or request changes for the WriteAgent to implement."
    ),
    llm=llm,
    tools=[review_report],
    can_handoff_to=["ResearchAgent","WriteAgent"],
)

## Running the Workflow

With our agents defined, we can create our `AgentWorkflow` and run it. Since this is a multi-agent system, instantiating it looks a little different. Instead of using the `from_tools_or_functions` helper, we instantiate it directly. It takes these arguments:
* an array of agents
* the name of the agent that it should start with (this will receive the initial `user_msg`)
* an `initial_state` which populates the context variable `state` we mentioned earlier

In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow

agent_workflow = AgentWorkflow(
    agents=[research_agent, write_agent, review_agent],
    root_agent=research_agent.name,
    initial_state={
        "research_notes": {},
        "report_content": "Not done.",
        "review": "Review required.",
    },
)

As the workflow is running, we will stream the events to get an idea of what is happening under the hood. We've added some decoration to the output to make this a little easier to read.

In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = agent_workflow.run(
    user_msg="""Analyze the theme of absurdity in Albert Camus's The Stranger. Summarize how Meursault's actions embody existentialist ideas, then connect them to Sartre's notion of freedom and responsibility. Put in multiple illustrative quotes. Add a references section in the end.""", 
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")

In [ ]:
from IPython.display import display, Markdown

state = await handler.ctx.get("state")
display(Markdown(state["report_content"]))


In [ ]:
from IPython.display import display, Markdown
display(Markdown(state["review"]))

# Building agentic workflows from scratch

As mentioned, `AgentWorkflow` is a pre-built `Workflow`, so you've already encountered many of the concepts you need to build a workflow from scratch.

You can easily build your own workflows directly to create multi-agent systems that are customized to your needs. There's a number of reasons you might want to branch out from AgentWorkflow to rolling your own workflow:

* **Fine-grained flow control**: Workflows allow looping, branching, parallel calls and map-reduce behavior. Workflows allow you to precisely specify how data gets passed around your system.

* **Structured inputs and outputs**: If you want your inputs and outputs to be richer than simple strings, or to kick off with more than just a `user_msg`, workflows allow you to do that.

* **Multimodality**: LlamaIndex can handle more than just text! You can handle images, audio and video as well.

* **Query planning**: a custom workflow can execute complex planning of how to run based on its inputs, rather than immediately jumping into a team.

* **Reflection**: A powerful technique for agents is their ability to examine their own output and decide whether it's sufficient, or if they need to try again. Using looping, you can implement this pattern.

## Creating a new workflow

Under the hood, Workflows are regular Python classes. They are defined as a series of `steps`, each of which receives certain classes of events and emits certain classes of events.

Here's the most basic form of a workflow, with a single step:

In [ ]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

class MyWorkflow(Workflow):
    @step
    async def my_step(self, ev: StartEvent) -> StopEvent:
        # do something here
        return StopEvent(result="Hello, world!")

This new `MyWorkflow` class:
* Uses the `@step` decorator to declare a function to be a step
* Has a single step called `my_step` which accepts a `StartEvent`. `StartEvent` is a special event which is always generated when a workflow first runs.
* `my_step` returns a `StopEvent`, which is another special event. When a `StopEvent` is emitted the workflow returns it and stops running.

You instantiate it and run it just like you ran the the `AgentWorkflow`:

In [ ]:
workflow = MyWorkflow(timeout=10, verbose=False)
result = await workflow.run()
print(result)

## Type annotations for steps

The type annotations (e.g. `ev: StartEvent` and `-> StopEvent`) are essential to the way Workflows work. The expected types determine what event types will trigger a step. Tools like the visualizer (see later) also rely on these annotations to determine what types are generated and therefore where control flow goes next.

## Aside: running a workflow in regular python

Workflows are async by default, so you use `await` to get the result of the `run` command. This will work fine in a notebook environment; in a vanilla python script you will need to import `asyncio` and wrap your code in an async function, like this:

```
async def main():
    w = MyWorkflow(timeout=10, verbose=False)
    result = await w.run()
    print(result)


if __name__ == "__main__":
    import asyncio
    asyncio.run(main())
```

Since we're in a notebook right now, we won't execute the above code as it won't work!

## Visualizing a workflow

A great feature of workflows is the built-in visualizer, which we will install now:


In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(MyWorkflow, filename="basic_workflow.html")

## Custom Events

Multiple steps are created by defining custom events that can be emitted by steps and trigger other steps. Let's define a simple 3-step workflow by defining two custom events, `FirstEvent` and `SecondEvent`. These classes can have any names and properties, but must inherit from `Event`:


In [ ]:
from llama_index.core.workflow import Event

class FirstEvent(Event):
    first_output: str

class SecondEvent(Event):
    second_output: str

In [ ]:
class MyWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent) -> FirstEvent:
        print(ev.first_input)
        return FirstEvent(first_output="First step complete.")

    @step
    async def step_two(self, ev: FirstEvent) -> SecondEvent:
        print(ev.first_output)
        return SecondEvent(second_output="Second step complete.")

    @step
    async def step_three(self, ev: SecondEvent) -> StopEvent:
        print(ev.second_output)
        return StopEvent(result="Workflow complete.")


w = MyWorkflow(timeout=10, verbose=False)
result = await w.run(first_input="Start the workflow.")
print(result)

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(MyWorkflow, filename="basic_workflow.html")

## Looping

However, there's not much point to a workflow if it just runs straight through! A key feature of Workflows is their enablement of branching and looping logic, more simply and flexibly than graph-based approaches. To enable looping, we'll create a new `LoopEvent` (LoopEvent is not special, any event can be used to loop).

In [ ]:
class LoopEvent(Event):
    first_input: str

In [ ]:
import random

class MyWorkflow(Workflow):

    # step_one will trigger on a StartEvent or a LoopEvent
    @step
    async def step_one(self, ev: StartEvent | LoopEvent) -> FirstEvent:
        print(ev.first_input)
        return FirstEvent(first_output="First step complete")

    # step two returns either a SecondEvent or a LoopEvent
    @step
    async def step_two(self, ev: FirstEvent) -> SecondEvent | LoopEvent:
        print(ev.first_output)
        if random.randint(0, 5) > 0:
            print("Bad thing happened")
            return LoopEvent(first_input="Back to step one.")
        else:
            print("Good thing happened")
            return SecondEvent(second_output="Second step complete.")

    @step
    async def step_three(self, ev: SecondEvent) -> StopEvent:
        print(ev.second_output)
        return StopEvent(result="Workflow complete.")


w = MyWorkflow(timeout=10, verbose=False)
result = await w.run(first_input="Start the workflow.")
print(result)

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(MyWorkflow, filename="basic_workflow.html")

## Branching

The same constructs that allow us to loop allow us to create branches. Here's a workflow that executes two different branches depending on an early decision:

In [ ]:
class BranchA1Event(Event):
    payload: str


class BranchA2Event(Event):
    payload: str


class BranchB1Event(Event):
    payload: str


class BranchB2Event(Event):
    payload: str


class BranchWorkflow(Workflow):
    @step
    async def start(self, ev: StartEvent) -> BranchA1Event | BranchB1Event:
        if random.randint(0, 1) == 0:
            print("Go to branch A")
            return BranchA1Event(payload="Branch A")
        else:
            print("Go to branch B")
            return BranchB1Event(payload="Branch B")

    @step
    async def step_a1(self, ev: BranchA1Event) -> BranchA2Event:
        print(ev.payload)
        return BranchA2Event(payload=ev.payload)

    @step
    async def step_b1(self, ev: BranchB1Event) -> BranchB2Event:
        print(ev.payload)
        return BranchB2Event(payload=ev.payload)

    @step
    async def step_a2(self, ev: BranchA2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="Branch A complete.")

    @step
    async def step_b2(self, ev: BranchB2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="Branch B complete.")

## Concurrent execution

The final form of flow control you can implement in workflows is concurrent execution. This allows you to efficiently run long-running tasks in parallel, and gather them together when they are needed. This can also let you perform map-reduce style tasks.

To do this, we'll be using the `Context` object that we already encountered when working with `AgentWorkflow`. The `Context` is available to every step in a workflow: to access it, declare it as an argument to your step and it will be automatically populated.

In this example, we use a new method, `Context.send_event` rather than returning an event. This allows us to emit multiple events in parallel rather than returning just one as we have previously.

In [ ]:
import asyncio

class StepTwoEvent(Event):
    query: str

class ParallelFlow(Workflow):
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> StepTwoEvent:
        ctx.send_event(StepTwoEvent(query="Query 1"))
        ctx.send_event(StepTwoEvent(query="Query 2"))
        ctx.send_event(StepTwoEvent(query="Query 3"))

    @step(num_workers=4)
    async def step_two(self, ctx: Context, ev: StepTwoEvent) -> StopEvent:
        print("Running slow query ", ev.query)
        await asyncio.sleep(random.randint(1, 5))

        return StopEvent(result=ev.query)

In [ ]:
w = ParallelFlow(timeout=10, verbose=False)
result = await w.run(message="Start the workflow.")
print(result)

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(BranchWorkflow, filename="basic_workflow.html")

As you can see, it executes all 3 queries. The first one to complete emits a StopEvent, at which point the workflow halts without waiting for the other 2 events.

## Collecting events

But what if we do want the output of all 3 events? Another method, `Context.collect_events`, exists for that purpose:

In [ ]:
class StepThreeEvent(Event):
    result: str

class ConcurrentFlow(Workflow):
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> StepTwoEvent:
        ctx.send_event(StepTwoEvent(query="Query 1"))
        ctx.send_event(StepTwoEvent(query="Query 2"))
        ctx.send_event(StepTwoEvent(query="Query 3"))

    @step(num_workers=4)
    async def step_two(self, ctx: Context, ev: StepTwoEvent) -> StepThreeEvent:
        print("Running query ", ev.query)
        await asyncio.sleep(random.randint(1, 5))
        return StepThreeEvent(result=ev.query)

    @step
    async def step_three(self, ctx: Context, ev: StepThreeEvent) -> StopEvent:
        # wait until we receive 3 events
        result = ctx.collect_events(ev, [StepThreeEvent] * 3)
        if result is None:
            print("Not all events received yet.")
            return None

        # do something with all 3 results together
        print(result)
        return StopEvent(result="Done")

In [ ]:
w = ConcurrentFlow(timeout=10, verbose=False)
result = await w.run(message="Start the workflow.")
print(result)

In the above example, we emit three `StepTwoEvent`s which trigger `step_two`, each execution of which eventually emits a `StepThreeEvent`.

`step_three` gets triggered whenever a `StepThreeEvent` fires. What `collect_events` does is store the events in the context until it has collected the number and type of events specified in its second argument. In this case, we've told it to wait for 3 events.

If an event fires and `collect_events` hasn't yet seen the right number of events, it returns `None`, so we tell `step_three` to do nothing in that case. When `collect_events` receives the right number of events it returns them as an array, which you can see us printing in the final output. Note that in the array they are stored in the order they returned, not the order they were emitted.

To implement a map-reduce pattern, you would split your task up into as many steps as necessary, and use `Context` to store that number with `ctx.set("num_events", some_number)`. Then in `step_three` you would wait for the number stored in the context using `await ctx.get("num_events")`. So you don't need to know in advance exactly how many concurrent steps you're taking.

## Collecting different event types

We don't just have to wait for multiple events of the same kind. In this example, we'll emit 3 totally different events and collect them at the end.

In [ ]:
class StepAEvent(Event):
    query: str

class StepACompleteEvent(Event):
    result: str

class StepBEvent(Event):
    query: str

class StepBCompleteEvent(Event):
    result: str

class StepCEvent(Event):
    query: str

class StepCCompleteEvent(Event):
    result: str

class ConcurrentFlow(Workflow):
    @step
    async def start(
        self, ctx: Context, ev: StartEvent
    ) -> StepAEvent | StepBEvent | StepCEvent:
        ctx.send_event(StepAEvent(query="Query 1"))
        ctx.send_event(StepBEvent(query="Query 2"))
        ctx.send_event(StepCEvent(query="Query 3"))

    @step
    async def step_a(self, ctx: Context, ev: StepAEvent) -> StepACompleteEvent:
        print("Doing something A-ish")
        return StepACompleteEvent(result=ev.query)

    @step
    async def step_b(self, ctx: Context, ev: StepBEvent) -> StepBCompleteEvent:
        print("Doing something B-ish")
        return StepBCompleteEvent(result=ev.query)

    @step
    async def step_c(self, ctx: Context, ev: StepCEvent) -> StepCCompleteEvent:
        print("Doing something C-ish")
        return StepCCompleteEvent(result=ev.query)

    @step
    async def step_three(
        self,
        ctx: Context,
        ev: StepACompleteEvent | StepBCompleteEvent | StepCCompleteEvent,
    ) -> StopEvent:
        print("Received event ", ev.result)

        # wait until we receive 3 events
        events = ctx.collect_events(
            ev,
            [StepCCompleteEvent, StepACompleteEvent, StepBCompleteEvent],
        )
        if (events is None):
            return None

        # do something with all 3 results together
        print("All events received: ", events)
        return StopEvent(result="Done")

In [ ]:
w = ConcurrentFlow(timeout=10, verbose=False)
result = await w.run(message="Start the workflow.")
print(result)

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(ConcurrentFlow, filename="basic_workflow.html")

## Streaming

We already saw `AgentWorkflow` emit a stream of events that we could handle and filter when we run the event. Let's see how those get emitted in this example, using `Context.write_event_to_stream`:

In [ ]:
class FirstEvent(Event):
    first_output: str

class SecondEvent(Event):
    second_output: str
    response: str

class TextEvent(Event):
    delta: str

class ProgressEvent(Event):
    msg: str

class MyWorkflow(Workflow):
    @step
    async def step_one(self, ctx: Context, ev: StartEvent) -> FirstEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step one is happening"))
        return FirstEvent(first_output="First step complete.")

    @step
    async def step_two(self, ctx: Context, ev: FirstEvent) -> SecondEvent:
        llm = OpenAI(model="gpt-4.1-mini", api_key=OPENAI_API_KEY) # the OpenAI key we set up at the beginning
        generator = await llm.astream_complete(
            "Please give me the first 50 words of Moby Dick, a book in the public domain."
        )
        async for response in generator:
            # Allow the workflow to stream this piece of response
            ctx.write_event_to_stream(TextEvent(delta=response.delta))
        return SecondEvent(
            second_output="Second step complete, full response attached",
            response=str(response),
        )

    @step
    async def step_three(self, ctx: Context, ev: SecondEvent) -> StopEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step three is happening"))
        return StopEvent(result="Workflow complete.")

In [ ]:
workflow = MyWorkflow(timeout=30, verbose=False)
handler = workflow.run(first_input="Start the workflow.")

async for ev in handler.stream_events():
    if isinstance(ev, ProgressEvent):
        print("\n"+ev.msg)
    if isinstance(ev, TextEvent):
        print(ev.delta, end="")

final_result = await handler
print("Final result = ", final_result)

# Putting together a multi-agent system

Now we've seen all the ways workflows can work, let's construct our own multi-agent system! This will be our Deep Research multi-agent system.

It will have three agents:
* `QuestionAgent` which accepts a research topic and generates a bunch of questions
* `AnswerAgent` which answers a specific question (we'll need to call it many times)
* `ReportAgent` which aggregates all the answers and generates a report.

We'll create these as `FunctionAgent`s, the same as we did when creating agents for `AgentWorkflow`.

In [7]:
question_agent = FunctionAgent(
    tools=[],
    llm=llm,
    verbose=False,
    system_prompt="""You are part of a deep research system.
      Given a research topic, you should come up with a bunch of questions
      that a separate agent will answer in order to write a comprehensive
      report on that topic. To make it easy to answer the questions separately,
      you should provide the questions one per line. Don't include markdown
      or any preamble in your response, just a list of questions."""
)

answer_agent = FunctionAgent(
    tools=[search_local_corpus ],
    llm=llm,
    verbose=False,
    system_prompt="""You are part of a deep research system.
      Given a specific question, your job is to come up with a deep answer
      to that question, which will be combined with other answers on the topic
      into a comprehensive report. You can search the web to get information
      on the topic, as many times as you need."""
)

report_agent = FunctionAgent(
    tools=[],
    llm=llm,
    verbose=False,
    system_prompt="""You are part of a deep research system.
      Given a set of answers to a set of questions, your job is to combine
      them all into a comprehensive report on the topic."""
)

NameError: name 'FunctionAgent' is not defined

The Workflow we'll need to handle this task needs to do a few things:
* Accept the topic and pass it to the QuestionAgent
* Take all the answers from the QuestionAgent and split them up, firing off one AnswerAgent for each question
* Aggregate all the questions and answers from the AnswerAgents
* Generate a single report from them

In [ ]:
class GenerateEvent(Event):
    research_topic: str

class QuestionEvent(Event):
    question: str

class AnswerEvent(Event):
    question: str
    answer: str

class ProgressEvent(Event):
    msg: str

class DeepResearchWorkflow(Workflow):

    @step
    async def setup(self, ctx: Context, ev: StartEvent) -> GenerateEvent:
        self.question_agent = ev.question_agent
        self.answer_agent = ev.answer_agent
        self.report_agent = ev.report_agent

        ctx.write_event_to_stream(ProgressEvent(msg="Starting research"))

        return GenerateEvent(research_topic=ev.research_topic)

    @step
    async def generate_questions(self, ctx: Context, ev: GenerateEvent) -> QuestionEvent:

        await ctx.set("research_topic", ev.research_topic)
        ctx.write_event_to_stream(ProgressEvent(msg=f"Research topic is {ev.research_topic}"))

        result = await self.question_agent.run(user_msg=f"""Generate some questions
          on the topic <topic>{ev.research_topic}</topic>.""")

        # some basic string manipulation to get separate questions
        lines = str(result).split("\n")
        questions = [line.strip() for line in lines if line.strip() != ""]

        # record how many answers we're going to need to wait for
        await ctx.set("total_questions", len(questions))

        # fire off multiple Answer Agents
        for question in questions:
            ctx.send_event(QuestionEvent(question=question))

    @step
    async def answer_question(self, ctx: Context, ev: QuestionEvent) -> AnswerEvent:

        result = await self.answer_agent.run(user_msg=f"""Research the answer to this
          question: <question>{ev.question}</question>. You can use web
          search to help you find information on the topic, as many times
          as you need. Return just the answer without preamble or markdown.""")

        ctx.write_event_to_stream(ProgressEvent(msg=f"""Received question {ev.question}
            Came up with answer: {str(result)}"""))

        return AnswerEvent(question=ev.question,answer=str(result))

    @step
    async def write_report(self, ctx: Context, ev: AnswerEvent) -> StopEvent:

        research = ctx.collect_events(ev, [AnswerEvent] * await ctx.get("total_questions"))
        # if we haven't received all the answers yet, this will be None
        if research is None:
            ctx.write_event_to_stream(ProgressEvent(msg="Collecting answers..."))
            return None

        ctx.write_event_to_stream(ProgressEvent(msg="Generating report..."))

        # aggregate the questions and answers
        all_answers = ""
        for q_and_a in research:
            all_answers += f"Question: {q_and_a.question}\nAnswer: {q_and_a.answer}\n\n"

        # prompt the report
        result = await self.report_agent.run(user_msg=f"""You are part of a deep research system.
          You have been given a complex topic on which to write a report:
          <topic>{await ctx.get("research_topic")}.

          Other agents have already come up with a list of questions about the
          topic and answers to those questions. Your job is to write a clear,
          thorough report that combines all the information from those answers.

          Here are the questions and answers:
          <questions_and_answers>{all_answers}</questions_and_answers>""")

        return StopEvent(result=str(result))

In [ ]:
workflow = DeepResearchWorkflow(timeout=600)
handler = workflow.run(
    research_topic="History of San Francisco",
    question_agent=question_agent,
    answer_agent=answer_agent,
    report_agent=report_agent
)

async for ev in handler.stream_events():
    if isinstance(ev, ProgressEvent):
        print(ev.msg)

final_result = await handler
print("==== The report ====")
print(final_result)

In [ ]:
display(Markdown(final_result))

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(DeepResearchWorkflow, filename="basic_workflow.html")

## Reflection

We now have a deep research agent! It comprehensively researches a topic for us before providing a detailed answer. But we can do better!

LLMs are capable of self-reflection: they can read their own work, critique it, and provide feedback, allowing them to take a second try when they fall short.

Let's add reflection to our deep research agent! This will involve several changes:

* In `research` we'll store the research into the context, since we might need to use it multiple times
* We'll tell `write` that it can be triggered by a `RewriteEvent` in addition to a `WriteEvent`
* If it's a `RewriteEvent` we'll add the review as feedback to the prompt
* `review` will be changed to optionally emit a `RewriteEvent`
* We'll get the LLM to decide if the review returned by the agent is a "bad" or "good" review

In [ ]:
class FeedbackEvent(Event):
    feedback: str

class ReviewEvent(Event):
    report: str

review_agent = FunctionAgent(
    tools=[],
    llm=llm,
    verbose=False,
    system_prompt="""You are part of a deep research system.
      Your job is to review a report that's been written and suggest
      questions that could have been asked to produce a more comprehensive
      report than the current version, or to decide that the current
      report is comprehensive enough."""
)

class DeepResearchWithReflectionWorkflow(Workflow):

    @step
    async def setup(self, ctx: Context, ev: StartEvent) -> GenerateEvent:
        self.question_agent = ev.question_agent
        self.answer_agent = ev.answer_agent
        self.report_agent = ev.report_agent
        self.review_agent = ev.review_agent
        self.review_cycles = 0

        ctx.write_event_to_stream(ProgressEvent(msg="Starting research"))

        return GenerateEvent(research_topic=ev.research_topic)

    @step
    async def generate_questions(self, ctx: Context, ev: GenerateEvent | FeedbackEvent) -> QuestionEvent:

        await ctx.set("research_topic", ev.research_topic)
        ctx.write_event_to_stream(ProgressEvent(msg=f"Research topic is {ev.research_topic}"))

        prompt = f"""Generate some questions on the topic <topic>{ev.research_topic}</topic>."""

        if isinstance(ev, FeedbackEvent):
            ctx.write_event_to_stream(ProgressEvent(msg=f"Got feedback: {ev.feedback}"))
            prompt += f"""You have previously researched this topic and
                got the following feedback, consisting of additional questions
                you might want to ask: <feedback>{ev.feedback}</feedback>.
                Keep this in mind when formulating your questions."""

        result = await self.question_agent.run(user_msg=prompt)

        # some basic string manipulation to get separate questions
        lines = str(result).split("\n")
        questions = [line.strip() for line in lines if line.strip() != ""]

        # record how many answers we're going to need to wait for
        await ctx.set("total_questions", len(questions))

        # fire off multiple Answer Agents
        for question in questions:
            ctx.send_event(QuestionEvent(question=question))

    @step
    async def answer_question(self, ctx: Context, ev: QuestionEvent) -> AnswerEvent:

        result = await self.answer_agent.run(user_msg=f"""Research the answer to this
          question: <question>{ev.question}</question>. You can use web
          search to help you find information on the topic, as many times
          as you need. Return just the answer without preamble or markdown.""")

        ctx.write_event_to_stream(ProgressEvent(msg=f"""Received question {ev.question}
            Came up with answer: {str(result)}"""))

        return AnswerEvent(question=ev.question,answer=str(result))

    @step
    async def write_report(self, ctx: Context, ev: AnswerEvent) -> ReviewEvent:

        research = ctx.collect_events(ev, [AnswerEvent] * await ctx.get("total_questions"))
        # if we haven't received all the answers yet, this will be None
        if research is None:
            ctx.write_event_to_stream(ProgressEvent(msg="Collecting answers..."))
            return None

        ctx.write_event_to_stream(ProgressEvent(msg="Generating report..."))

        # aggregate the questions and answers
        all_answers = ""
        for q_and_a in research:
            all_answers += f"Question: {q_and_a.question}\nAnswer: {q_and_a.answer}\n\n"

        # prompt the report
        result = await self.report_agent.run(user_msg=f"""You are part of a deep research system.
          You have been given a complex topic on which to write a report:
          <topic>{await ctx.get("research_topic")}.

          Other agents have already come up with a list of questions about the
          topic and answers to those questions. Your job is to write a clear,
          thorough report that combines all the information from those answers.

          Here are the questions and answers:
          <questions_and_answers>{all_answers}</questions_and_answers>""")

        return ReviewEvent(report=str(result))

    @step
    async def review(self, ctx: Context, ev: ReviewEvent) -> StopEvent | FeedbackEvent:

        result = await self.review_agent.run(user_msg=f"""You are part of a deep research system.
          You have just written a report about the topic {await ctx.get("research_topic")}.
          Here is the report: <report>{ev.report}</report>
          Decide whether this report is sufficiently comprehensive.
          If it is, respond with just the string "ACCEPTABLE" and nothing else.
          If it needs more research, suggest some additional questions that could
          have been asked.""")

        self.review_cycles += 1

        # either it's okay or we've already gone through 3 cycles
        if str(result) == "ACCEPTABLE" or self.review_cycles >= 3:
            return StopEvent(result=ev.report)
        else:
            ctx.write_event_to_stream(ProgressEvent(msg="Sending feedback"))
            return FeedbackEvent(
                research_topic=await ctx.get("research_topic"),
                feedback=str(result)
            )

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(DeepResearchWithReflectionWorkflow, filename="basic_workflow.html")

In [ ]:
workflow = DeepResearchWithReflectionWorkflow(timeout=600)
handler = workflow.run(
    research_topic="""
    History of San Francisco.
    """,
    question_agent=question_agent,
    answer_agent=answer_agent,
    report_agent=report_agent,
    review_agent=review_agent
)

async for ev in handler.stream_events():
    if isinstance(ev, ProgressEvent):
        print(ev.msg)

final_result = await handler
print("==== The report ====")
print(final_result)

In [ ]:
from IPython.display import display, Markdown

display(Markdown(final_result))